#OAuth Exercise

In this exercise we will try to scrape twitter data and do a tf-idf analysis on that (src-uwes twitter analysis). We will need OAuth authentication, and we will follow a similar approach as detailed in the yelp analysis notebook. 

In [32]:
import oauth2 as oauth
import urllib2 as urllib

We will now need twitter api access. The following steps as available online will help you set up your twitter account and access the live 1% stream.

1. Create a twitter account if you do not already have one.
2. Go to https://dev.twitter.com/apps and log in with your twitter credentials.
3. Click "Create New App"
4. Fill out the form and agree to the terms. Put in a dummy website if you don't have one you want to use.
5. On the next page, click the "API Keys" tab along the top, then scroll all the way down until you see the section "Your Access Token"
6. Click the button "Create My Access Token". You can Read more about Oauth authorization online. 

Save the details of api_key, api_secret, access_token_key, access_token_secret in your vaule directory and load it in the notebook as shown in yelpSample notebook.

In [33]:
import Twitter_Keys
TW_Keys=Twitter_Keys.getdict()

In [34]:
api_key = TW_Keys['api_key']#<get api key>
api_secret = TW_Keys['api_secret']  #<get api secret>
access_token_key = TW_Keys['access_token_key'] #<get your access token key here>"
access_token_secret = TW_Keys['access_token_secret'] #<get your access token secret here>


_debug = 0

oauth_token    = oauth.Token(key=access_token_key, secret=access_token_secret)
oauth_consumer = oauth.Consumer(key=api_key, secret=api_secret)

signature_method_hmac_sha1 = oauth.SignatureMethod_HMAC_SHA1()

http_method = "GET"

http_handler  = urllib.HTTPHandler(debuglevel=_debug)
https_handler = urllib.HTTPSHandler(debuglevel=_debug)

Below is a twitter request method which will use the above user logins to sign, and open a twitter stream request

In [35]:
def getTwitterStream(url, method, parameters):
    req = oauth.Request.from_consumer_and_token(oauth_consumer,
                                             token=oauth_token,
                                             http_method=http_method,
                                             http_url=url, 
                                             parameters=parameters)

    req.sign_request(signature_method_hmac_sha1, oauth_consumer, oauth_token)

    headers = req.to_header()

    if http_method == "POST":
        encoded_post_data = req.to_postdata()
    else:
        encoded_post_data = None
        url = req.to_url()

    opener = urllib.OpenerDirector()
    opener.add_handler(http_handler)
    opener.add_handler(https_handler)

    response = opener.open(url, encoded_post_data)

    return response


We can use the above function to request a response as follows

In [36]:
#Now we will test the above function for a sample data provided by twitter stream here -  
url = "https://stream.twitter.com/1/statuses/sample.json"

In [37]:
parameters = []
response = getTwitterStream(url, "GET", parameters)

Write a function which will take a url and return the top 10 lines returned by the twitter stream

** Note ** The response returned needs to be intelligently parsed to get the text data which correspond to actual tweets. This part can be done in a number of ways and you are encouraged to try different approaches to parse the response data.

In [38]:
response.read(1000)

'{"created_at":"Mon Dec 07 06:15:07 +0000 2015","id":673747547739975680,"id_str":"673747547739975680","text":"\\u3046\\u30fc\\u30fc\\u30fc\\u30fc\\u30fc\\u306f\\u3041\\u3042\\u3042\\u3042\\u3042\\u3042\\u3042\\u3042\\u3046\\u30fc\\u30fc\\u30fc\\u30fc\\u30fc\\u30fc\\u30fc\\u30fc\\u30fc\\u30fc\\u30fc\\u30fc\\u30fc\\u306f\\u3041\\u3042\\u3042\\u3042\\u3042\\u3042\\u3042\\u3042\\u3042\\u3042\\u3042\\uff01\\uff01\\uff01","source":"\\u003ca href=\\"http:\\/\\/twittbot.net\\/\\" rel=\\"nofollow\\"\\u003etwittbot.net\\u003c\\/a\\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":2257643376,"id_str":"2257643376","name":"\\u30b4\\u30ea\\u62bc\\u3057\\u6295\\u3052\\u592a\\u90ce","screen_name":"nagetarou_u_tan","location":null,"url":null,"description":"\\u304a\\u3063\\u307a\\u3051\\u30e9\\u30fc\\u6cb9\\u30bb\\u30f3\\u30bf\\u30fc\\u6240\\u5c5e\\u3002\\u56db\\u592

In [39]:
import re
re.search(r'(?<=,"text":).*(?=,"source)', response.read(1000).decode('utf-8')).group()

AttributeError: 'NoneType' object has no attribute 'group'

In [64]:
####modified the input of fetchData to include the number of requested tweets####
def fetchData(url,tweet_count=10):
    parameters = []
    top_tweet=[]
    
    if 'stream.twitter.com' in url:
        response = getTwitterStream(url, "GET", parameters)
        i=0
        for line in response:
            try:
                if i<tweet_count:
                    i+=1
                    top_tweet.append(json.loads(line)['text'])
                else:
                    break
            except:
                i-=1
                pass
        return top_tweet
    
    elif 'api.twitter.com' in url:
        url=url+"&count="+str(tweet_count)
        response = getTwitterStream(url, "GET", parameters)
        response_content=json.loads(response.read())['statuses']
        i=0
        content=[]
        for line in response_content:
            try:
                if i<tweet_count:
                    i+=1
                    top_tweet.append(line['text'])
                else:
                    break
            except:
                i-=1
                pass
        return top_tweet

In [41]:
url = "https://stream.twitter.com/1/statuses/sample.json"
top_10_tweet=fetchData(url)
print "Top 10 streamed tweets are:\n\n"
i=1
for tweet in top_10_tweet:
    print "tweet %d is:"%(i),tweet,"\n"
    i+=1

Top 10 streamed tweets are:


tweet 1 is: ユニフォーム泥だらけになって
腕も泥だらけになって
そんながんばってる姿が
とってもかっこいいと
改めて思った1日だった 

tweet 2 is: ｸｳｳ 

tweet 3 is: bwisit talaga kainis kapal talaga 

tweet 4 is: @SC_YBora90 siang ya 

tweet 5 is: @azuuuuuuuchan まさかの鴻巣だったフラワーバス 

tweet 6 is: RT @justink154: @DailySexSupply @SexualGif LOL 

tweet 7 is: RT @WORIDSTARHIPHOP: LMAO https://t.co/jgUetRqdJz 

tweet 8 is: RT @BieberBonerz: this video saved me OMG 😍😍 https://t.co/MQf85wPyBT 

tweet 9 is: Ah no. Mañana a las ocho estoy durmiendo. Me dan estas noticias a estas horas. 

tweet 10 is: RT @MalditangMayora: Yung nag-aalok ka ng pagkain pero secretly hoping na sana ayaw nila. 



In [ ]:
#We can also request twitter stream data for specific search parameters as follows
url= "https://api.twitter.com/1.1/search/tweets.json?q="+search_query

Call the fetchData function to fetch latest live stream data for following search queries and output the first 5 lines

1. "UCSD"
2. "Donald Trump"
3. "Syria"

In [43]:
import json
queries=["UCSD","Donald%20Trump","Syria"]
for query in queries:
    url= "https://api.twitter.com/1.1/search/tweets.json?q="+query
    top_10_tweet=fetchData(url)
    top_5_tweet=fetchData(url, tweet_count=5)
    print "Top 5 tweets about %s is:\n\n"%(query)
    for tweet in top_5_tweet:
        print tweet,'\n'
    print "\n\n"

Top 5 tweets about UCSD is:


ユニフォーム泥だらけになって
腕も泥だらけになって
そんながんばってる姿が
とってもかっこいいと
改めて思った1日だった 

ｸｳｳ 

bwisit talaga kainis kapal talaga 

@SC_YBora90 siang ya 

@azuuuuuuuchan まさかの鴻巣だったフラワーバス 

RT @justink154: @DailySexSupply @SexualGif LOL 

RT @WORIDSTARHIPHOP: LMAO https://t.co/jgUetRqdJz 

RT @BieberBonerz: this video saved me OMG 😍😍 https://t.co/MQf85wPyBT 

Ah no. Mañana a las ocho estoy durmiendo. Me dan estas noticias a estas horas. 

RT @MalditangMayora: Yung nag-aalok ka ng pagkain pero secretly hoping na sana ayaw nila. 




Top 5 tweets about Donald%20Trump is:


ユニフォーム泥だらけになって
腕も泥だらけになって
そんながんばってる姿が
とってもかっこいいと
改めて思った1日だった 

ｸｳｳ 

bwisit talaga kainis kapal talaga 

@SC_YBora90 siang ya 

@azuuuuuuuchan まさかの鴻巣だったフラワーバス 

RT @justink154: @DailySexSupply @SexualGif LOL 

RT @WORIDSTARHIPHOP: LMAO https://t.co/jgUetRqdJz 

RT @BieberBonerz: this video saved me OMG 😍😍 https://t.co/MQf85wPyBT 

Ah no. Mañana a las ocho estoy durmiendo. Me dan estas noticias a estas horas. 

RT @Mal

### TF-IDF###

tf–idf, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.It is among the most regularly used statistical tool for word cloud analysis. You can read more about it online (https://en.wikipedia.org/wiki/Tf%E2%80%93idf)

We base our analysis on the following

1. The weight of a term that occurs in a document is simply proportional to the term frequency
2. The specificity of a term can be quantified as an inverse function of the number of documents in which it occurs

For this question we will perform tf-idf analysis o the stream data we retrieve for a given search parameter. Perform the steps below

1. use the twitterreq function to search for the query "syria" and save the top 200 lines in the file twitterStream.txt
2. load the saved file and output the count of occurrences for each term. This will be your term frequency
3. Calculate the inverse document frequency for each of the term in the output above.
4. Divide the term frequency for each of the term by corresponding inverse document frequency.
5. Sort the terms in the descending order based on their term freq/inverse document freq scores 
6. Print the top 10 terms.

In [227]:
####Solving the problem using the fetchData function above and RESTful API####

url= "https://api.twitter.com/1.1/search/tweets.json?q=Syria"
top_200_tweet=fetchData(url, tweet_count=200)
with open('twitterStream.txt','wb') as ts:
    [ts.write(tweet.lower().encode('utf-8')+'\n') for tweet in top_200_tweet]

In [228]:
####Based on the clarification on piazza, I considered each tweet as one single documents and hence tf is the word
#frequency within each document
from collections import Counter
tf=[]
tweet_file=open('twitterStream.txt','rb')

for line in tweet_file:
    for char in '-.,"\n':
        line=line.replace(char,' ')
    c=Counter(line.split())
    tf.append(c)

tweet_file.close()

In [229]:
import numpy as np
from __future__ import division
idf_count={}
idf={}
for doc in range(len(tf)):
    for term in set(tf[doc]):
        idf_count[(doc,term)]=1
        tweet_file=open('twitterStream.txt','rb')
        for line in tweet_file:
            if term in line:
                idf_count[(doc,term)]+=1
        tweet_file.close()
        idf[(doc,term)]=np.log(len(tf)/idf_count[(doc,term)])

In [230]:
tf_idf={}
for doc in range(len(tf)):
    for term in set(tf[doc]):
        tf_idf[(term,tf[doc][term],idf[(doc,term)])]=tf[doc][term] * idf[(doc,term)]

In [231]:
import operator
sorted_tf_idf=sorted(tf_idf.items(), key=operator.itemgetter(1), reverse=True)

In [232]:
from prettytable import PrettyTable

pt = PrettyTable(field_names=['tf', 'idf', 'td-idf score']) 
[pt.add_row((tf_idf_row[0][1],tf_idf_row[0][2],tf_idf_row[1])) for tf_idf_row in sorted_tf_idf ]
pt.add_column('term',[tf_idf_row[0][0] for tf_idf_row in sorted_tf_idf])
pt.padding_width = 1
pt.align='c' # Set column alignment
print pt[0:10]

+----+---------------+---------------+----------+
| tf |      idf      |  td-idf score |   term   |
+----+---------------+---------------+----------+
| 3  |  3.0169344812 | 9.05080344361 |  attack  |
| 2  | 3.57655026914 | 7.15310053828 |  bombed  |
| 2  | 3.35340671783 | 6.70681343565 |   like   |
| 2  | 3.35340671783 | 6.70681343565 |   can    |
| 2  |  3.0169344812 | 6.03386896241 | petition |
| 2  |  3.0169344812 | 6.03386896241 |  signed  |
| 2  | 2.88340308858 | 5.76680617716 |   ban    |
| 2  | 2.88340308858 | 5.76680617716 |  #isis   |
| 2  | 2.76562005292 | 5.53124010585 |   any    |
| 2  | 2.47793798047 | 4.95587596094 |   and    |
+----+---------------+---------------+----------+


In [236]:
####Solving the same problem using Tweepy library####

from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener

import sys 
sys.path.append('/Users/phesami/Documents/DSE/phesami/DSE200/day_5_mining_the_Social_web/exercises/')
import Twitter_Keys
#consumer key, consumer secret, access token, access secret.
atoken,csecret,ckey, asecret=Twitter_Keys.getkeys()

class listener(StreamListener):
    
    def __init__(self):
        super(listener, self).__init__()
        self.counter=1
        self.tweet_count=200
        self.ts=open('twitterStream_Tweepy.txt','wb')

    def on_status(self, status):
        while self.counter<self.tweet_count:
            self.ts.write(status.text.lower().encode('utf-8')+'\n')
            self.counter+=1
            return True
        self.ts.close()
        return False

    def on_error(self, status):
        print "error",status


auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["syria"])

In [237]:
import numpy as np
from __future__ import division
from collections import Counter
tf=[]
tweet_file=open('twitterStream_Tweepy.txt','rb')

for line in tweet_file:
    c=Counter(line.split())
    tf.append(c)

tweet_file.close()

idf_count={}
idf={}
for doc in range(len(tf)):
    for term in set(tf[doc]):
        idf_count[(doc,term)]=1
        tweet_file=open('twitterStream_Tweepy.txt','rb')
        for line in tweet_file:
            if term in line:
                idf_count[(doc,term)]+=1
        tweet_file.close()
        idf[(doc,term)]=np.log(len(tf)/idf_count[(doc,term)])
        
        
tf_idf={}
for doc in range(len(tf)):
    for term in set(tf[doc]):
        tf_idf[(term,tf[doc][term],idf[(doc,term)])]=tf[doc][term] * idf[(doc,term)]
        
import operator
sorted_tf_idf=sorted(tf_idf.items(), key=operator.itemgetter(1), reverse=True)

from prettytable import PrettyTable

pt = PrettyTable(field_names=['tf', 'idf', 'td-idf score']) 
[pt.add_row((tf_idf_row[0][1],tf_idf_row[0][2],tf_idf_row[1])) for tf_idf_row in sorted_tf_idf ]
pt.add_column('term',[tf_idf_row[0][0] for tf_idf_row in sorted_tf_idf])
pt.padding_width = 1
pt.align='c' # Set column alignment
print pt[0:10]

+----+---------------+---------------+------------+
| tf |      idf      |  td-idf score |    term    |
+----+---------------+---------------+------------+
| 2  | 5.04342511692 | 10.0868502338 |  #germany  |
| 2  | 5.04342511692 | 10.0868502338 |   labour   |
| 2  | 5.04342511692 | 10.0868502338 | supporting |
| 2  | 4.35027793636 | 8.70055587272 | jerusalem  |
| 2  | 4.35027793636 | 8.70055587272 |  deserves  |
| 2  | 4.35027793636 | 8.70055587272 |    post    |
| 2  | 4.35027793636 | 8.70055587272 |   called   |
| 2  | 4.12713438505 | 8.25426877009 |   their    |
| 2  | 3.79066214842 | 7.58132429685 |   &amp;    |
| 2  | 3.25166564769 | 6.50333129538 |    you     |
+----+---------------+---------------+------------+
